In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path
import os
import autogluon
from autogluon.tabular import TabularDataset,TabularPredictor
from sklearn.model_selection import train_test_split
import utils
import geopandas as gpd

In [2]:
current_dir = Path.cwd()
data_dir = current_dir.parent / "data"

In [3]:
train_df = pd.read_csv(os.path.join("..", "data", "train", "train.csv"))
drop = [
    "reform_place_other",
    "reform_common_area",
    "parking_keiyaku",
    "est_other_distance",
    "building_name_ruby",
    "name_ruby",
    "school_ele_code",
    "school_jun_code",
    "money_hoshou_company",
    "free_rent_duration",
    "free_rent_gen_timing",
    "post2",
    "nl",
    "el",
    "unit_name",
    "money_sonota_str2",
    "basement_floor_count",
    "money_sonota3",
    "land_seigen",
    "snapshot_land_shidou",
    "money_sonota2",
    "bank_distance",
]
train_df = train_df.drop(columns=drop)
len(train_df)

C:\Users\it160\AppData\Local\Temp\ipykernel_12364\2604508089.py:1: DtypeWarning: Columns (63) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(os.path.join("..", "data", "train", "train.csv"))


363924

In [4]:
gdf_json = gpd.read_file(os.path.join('..', 'data', 'train', 'L01-24.geojson'))#[land_price_cols]
gdf_json2 = gpd.read_file(os.path.join('..', 'data', 'train', 'L02-24.geojson'))#[land_price_cols]
map1=utils.generate_year_mapping(1982,'L01_062',104-62,0)
map2=utils.generate_year_mapping(1982,'L02_055',97-55,0)

In [5]:
# L02_098～L02_134のカラムを削除
drop_cols = [f'L01_{i:03d}' for i in range(105, 147)]
drop_cols.extend([f'L01_{i:03d}' for i in range(10, 27)])
drop_cols.extend([f'L01_{i:03d}' for i in range(62, 98)])
drop_cols.extend([f'L01_{i:03d}' for i in range(51, 57)])
drop_cols.append('L01_103')
gdf_json = gdf_json.drop(columns=[col for col in drop_cols if col in gdf_json.columns], errors='ignore')

drop_cols = [c for c in gdf_json.columns if c.startswith('L01_') and int(c.split('_')[1]) >= 51]
gdf_json = gdf_json.drop(columns=drop_cols, errors='ignore')

# L02については72-97のカラムのみを抽出
keep_cols = [f'L02_{i:03d}' for i in range(90, 98)]
keep_cols.append('geometry')  # geometry列も保持
gdf_json2 = gdf_json2[[col for col in keep_cols if col in gdf_json2.columns]]

drop_cols = [c for c in gdf_json2.columns if c.startswith('L02_') and int(c.split('_')[1]) >= 51]
gdf_json2 = gdf_json2.drop(columns=drop_cols, errors='ignore')

In [6]:
gdf_json.columns

Index(['L01_001', 'L01_002', 'L01_003', 'L01_004', 'L01_005', 'L01_006',
       'L01_007', 'L01_008', 'L01_009', 'L01_027', 'L01_028', 'L01_029',
       'L01_030', 'L01_031', 'L01_032', 'L01_033', 'L01_034', 'L01_035',
       'L01_036', 'L01_037', 'L01_038', 'L01_039', 'L01_040', 'L01_041',
       'L01_042', 'L01_043', 'L01_044', 'L01_045', 'L01_046', 'L01_047',
       'L01_048', 'L01_049', 'L01_050', 'geometry'],
      dtype='object')

In [6]:
def merge_lag_features(df,data,join_key,prefix,map):
    gdf=utils.convert_to_geodataframe(df)
    data[join_key]=range(0, len(data))
    gdf=utils.nearest_merge(gdf,data[[join_key,"geometry"]],threshold=1000)
    df_lag=utils.merge_lag_features(gdf,data,join_key,prefix,map)
    df_lag=df_lag.drop(columns=[join_key])
    return df_lag

In [7]:
train_gdf=utils.convert_to_geodataframe(train_df)
source_df=merge_lag_features(train_gdf,gpd.read_file(os.path.join('..', 'data', 'train', 'L01-24.geojson')),"ID","_kouji",map1)
source_df=merge_lag_features(source_df,gpd.read_file(os.path.join('..', 'data', 'train', 'L02-24.geojson')),"ID","_tyousa",map2)
source_df=source_df.drop(columns=["distance"])
source_df=utils.nearest_merge(source_df,gdf_json,threshold=1000)

In [8]:
len(train_df),len(source_df)

(363924, 363924)

In [8]:
map3=utils.generate_year_mapping(2011,'S01_008',14,0)
rosen_gdf = gpd.read_file(os.path.join('..', 'data', 'train', 'S12-24_NumberOfPassengers.geojson'))
rosen=rosen_gdf.groupby(by="S12_001g").agg(
    S01_020=("S12_057","sum"),
    S01_019=("S12_053","sum"),
    S01_018=("S12_049","sum"),
    S01_017=("S12_045","sum"),
    S01_016=("S12_041","sum"),
    S01_015=("S12_037","sum"),
    S01_014=("S12_033","sum"),
    S01_013=("S12_029","sum"),
    S01_012=("S12_025","sum"),
    S01_011=("S12_021","sum"),
    S01_010=("S12_017","sum"),
    S01_009=("S12_013","sum"),
    S01_008=("S12_009","sum"),
    eki_name1=("S12_001","first"),
    geometry=("geometry","first")
)
rosen["S01_021"]=np.nan
#rosen=rosen.replace({0:np.nan})
rosen["ID"]=range(0, len(rosen))
rosen=gpd.GeoDataFrame(rosen,geometry="geometry",crs="EPSG:6677")
rosen.head()
# result_df=utils.merge_lag_features(source_df,rosen,"eki_name1","_eki1",map3)
# rosen=rosen.rename(columns={"eki_name1":"eki_name2"})
# source_df=utils.merge_lag_features(source_df,rosen,"eki_name2","_eki2",map3)
# source_df.head()

,S01_020,S01_019,S01_018,S01_017,S01_016,S01_015,S01_014,S01_013,S01_012,S01_011,S01_010,S01_009,S01_008,eki_name1,geometry,S01_021,ID
S12_001g,,,,,,,,,,,,,,,,,
000001,114.0,0.0,0.0,88.0,166.0,166.0,168.0,208.0,214.0,190.0,200.0,212.0,212.0,稚内,"MULTILINESTRING ((141.677 45.417, 141.677 45.4...",NaN,0
000002,82.0,0.0,0.0,78.0,128.0,138.0,152.0,166.0,170.0,162.0,170.0,204.0,222.0,南稚内,"MULTILINESTRING ((141.683 45.398, 141.683 45.3...",NaN,1
000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,抜海,"MULTILINESTRING ((141.645 45.317, 141.645 45.3...",NaN,2
000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,2.0,0.0,勇知,"MULTILINESTRING ((141.685 45.26, 141.685 45.259))",NaN,3
000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,4.0,10.0,14.0,兜沼,"MULTILINESTRING ((141.697 45.22, 141.698 45.22))",NaN,4


In [28]:
train_df[train_df["eki_name1"].notna()]

,target_ym,money_room,building_id,building_status,building_create_date,building_modify_date,building_type,building_name,homes_building_name,homes_building_name_ruby,...,school_jun_name,school_jun_distance,convenience_distance,super_distance,hospital_distance,park_distance,drugstore_distance,shopping_street_distance,est_other_name,statuses
0,201901,13980000,206271,1,2014-06-27 21:09:41,2019-01-24 00:10:08,4,NaN,桑名市東正和台7丁目10-11,NaN,...,正和中学校,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,210101/220701/220601/230401/310501/210301/210202
1,201901,24480000,83315,1,2014-06-27 21:09:43,2019-04-03 00:10:08,4,NaN,桑名市松ノ木5丁目,NaN,...,陵成中学校,1100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,210101/220701/220601/220801/230601/250201/2103...
2,201901,24480000,140201,1,2014-06-27 21:09:43,2020-06-06 00:10:11,4,NaN,桑名市松ノ木七丁目,NaN,...,陵成中学校,1200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,210101/220701/220601/230401/220801/310501/2306...
3,201901,16300000,216551,1,2014-06-27 19:24:11,2019-04-26 00:10:08,4,NaN,中古戸建 知多市八幡字荒井,NaN,...,八幡中学校,1400.0,NaN,NaN,NaN,NaN,NaN,NaN,コンビニ,210101/290901/230401/210301/210401
4,201901,18800000,134968,1,2014-06-27 19:24:12,2025-09-15 02:49:23,1,ロイヤル知多寺本,ロイヤル知多寺本,NaN,...,八幡中学校,1410.0,NaN,NaN,NaN,NaN,1060.0,NaN,駅,220801/220401/250101/220701/321101/220601/2304...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363919,202207,14300000,182260,1,2014-06-27 21:09:33,2025-10-04 00:45:08,1,藤和シティコープ桑名駅前,藤和シティコープ桑名駅前II,NaN,...,NaN,NaN,290.0,500.0,530.0,NaN,NaN,NaN,NaN,230101/220301/220401/220601/230401/210201/2201...
363920,202207,18900000,118825,1,2014-06-27 21:09:35,2025-09-21 00:45:08,1,プラザ桑名,プラザ桑名 リノベ 3LDK 駅徒歩2分,NaN,...,NaN,NaN,80.0,NaN,330.0,NaN,650.0,NaN,NaN,220801/230601/230101/220301/220401/230801/2207...
363921,202207,15900000,92766,1,2014-06-27 21:09:35,2025-09-16 00:45:08,1,エスポア桑名2,エスポア桑名II 1102,NaN,...,桑名市立光風中学校,430.0,430.0,740.0,300.0,NaN,510.0,NaN,NaN,220801/230601/230101/220301/220401/230801/2207...
363922,202207,47800000,212417,1,2014-06-27 19:24:11,2023-10-23 09:49:37,4,NaN,南粕谷2丁目 中古戸建,NaN,...,知多市立旭南中学校,2128.0,817.0,823.0,521.0,2652.0,NaN,3693.0,NaN,253701/250301/290501/220801/230601/220301/2204...


In [9]:
result_gdf = utils.join_nearest_with_conditions(source_df, rosen[["ID","geometry","eki_name1"]].reset_index(drop=True), match_col='eki_name1', max_dist=100000.0)
result_gdf=result_gdf.drop(columns=["index__target","eki_name1_"])
result_gdf=result_gdf.rename(columns={"eki_name1__target":"eki_name1"})
result_gdf=utils.merge_lag_features(result_gdf,rosen,"ID","_eki1",map3)
result_gdf=result_gdf.drop(columns=["ID"])
rosen=rosen.rename(columns={"eki_name1":"eki_name2"})
result_gdf = utils.join_nearest_with_conditions(result_gdf, rosen[["ID","geometry","eki_name2"]].reset_index(drop=True), match_col='eki_name2', max_dist=100000.0)
result_gdf=result_gdf.drop(columns=["index__target","eki_name2_"])
result_gdf=result_gdf.rename(columns={"eki_name2__target":"eki_name2"})
result_gdf=utils.merge_lag_features(result_gdf,rosen,"ID","_eki2",map3)
result_gdf=result_gdf.drop(columns=["ID"])

In [30]:

len(train_df),len(result_gdf)
result_gdf[result_gdf["lag_1_val__eki2"].notna()]

,target_ym,money_room,building_id,building_status,building_create_date,building_modify_date,building_type,building_name,homes_building_name,homes_building_name_ruby,...,lag_2_val__eki1,lag_3_val__eki1,lag_4_val__eki1,lag_5_val__eki1,eki_name2,lag_1_val__eki2,lag_2_val__eki2,lag_3_val__eki2,lag_4_val__eki2,lag_5_val__eki2
35,201901,11800000,79392,1,2014-06-27 21:12:09,2025-09-15 01:17:12,1,シティーナ神立カメリア館B棟,シティーナ神立 カメリア館B棟,NaN,...,10844.0,11064.0,10812.0,10990.0,土浦,32248.0,32114.0,32446.0,31856.0,32472.0
39,201901,8000000,2524,1,2014-06-27 21:12:09,2019-05-03 00:10:08,4,NaN,かすみがうら市下稲吉 戸建,カスミガウラシシモイナヨシ,...,10844.0,11064.0,10812.0,10990.0,神立,11080.0,10844.0,11064.0,10812.0,10990.0
41,201901,36000000,33061,1,2014-06-27 21:12:32,2019-09-20 00:10:10,4,NaN,あきる野市伊奈 中古一戸建 全1棟,NaN,...,5206.0,5268.0,5260.0,5362.0,武蔵五日市,8752.0,8854.0,9056.0,9194.0,9248.0
42,201901,18800000,125947,9,2014-06-27 21:12:33,2024-06-04 12:02:55,4,NaN,あきる野市牛沼 陶芸工房付き中古住宅 増改築多数,NaN,...,14444.0,14506.0,14452.0,14960.0,東秋留,9438.0,9598.0,9632.0,9662.0,9814.0
45,201901,18500000,1878,1,2014-06-27 21:12:33,2019-04-16 00:10:09,4,NaN,あきる野市小川 中古住宅,NaN,...,9598.0,9632.0,9662.0,9814.0,熊川,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363908,202207,16000000,46802,1,2014-06-27 21:09:17,2024-06-04 13:11:02,4,NaN,船橋市藤原3丁目,NaN,...,27152.0,38318.0,38318.0,37082.0,馬込沢,22243.0,20928.0,26529.0,26787.0,26786.0
363909,202207,17990000,70861,1,2014-06-27 21:09:18,2025-09-15 01:34:57,1,西船橋スカイハイツ,西船橋スカイハイツ,ニシフナバシスカイハイツ,...,27152.0,38318.0,38318.0,37082.0,西船橋,535691.0,511021.0,693803.0,870580.0,686351.0
363910,202207,8800000,152376,1,2014-06-27 21:09:18,2023-05-15 00:10:05,4,NaN,船橋市藤原5丁目,NaN,...,20928.0,26529.0,26787.0,26786.0,船橋法典,29198.0,27152.0,38318.0,38318.0,37082.0
363912,202207,8900000,10045,1,2014-06-27 21:09:19,2025-09-15 01:34:57,1,牡鹿台ハイツ8,牡鹿台ハイツ,NaN,...,20928.0,26529.0,26787.0,26786.0,船橋,318306.0,297404.0,392768.0,393901.0,391574.0


In [17]:
counts = rosen["eki_name1"].value_counts()
duplicates = counts[counts > 1]

print(duplicates)

eki_name1
市役所前    8
県庁前     7
住吉      6
高田      6
大町      6
       ..
難波      2
高井田     2
西宮      2
霞ヶ丘     2
郡元      2
Name: count, Length: 427, dtype: int64


In [9]:
# pandasの表示設定
pd.set_option('display.max_columns', None)  # すべての列を表示
pd.set_option('display.max_rows', None)     # すべての行を表示（オプション）
#pd.set_option('display.width', None)        # 行の折り返しを防ぐ
#pd.set_option('display.max_colwidth', None) # 列の内容を省略しない

In [12]:
len(train_gdf.columns)

186

In [15]:
# distance列がNULLとなっている割合を計算
null_ratio = train_gdf['distance'].isnull().sum() / len(train_gdf)
print(f"distance列のNULL割合: {null_ratio:.2%} ({train_gdf['distance'].isnull().sum()}/{len(train_gdf)})")


distance列のNULL割合: 12.05% (43843/363924)


In [11]:
type(source_df)

geopandas.geodataframe.GeoDataFrame

In [10]:
source_df=result_gdf.loc[:, ~result_gdf.columns.duplicated()]
source_df=source_df.drop(columns=["target_ym"])

In [11]:
# GeoDataFrameをDataFrameに変換(geometry列を削除)
source_df["land_price_x"]=source_df.geometry.x
source_df["land_price_y"]=source_df.geometry.y
source_df = pd.DataFrame(source_df.drop(columns=[col for col in source_df.columns if 'geometry' in col]))

In [ ]:
for s in source_df.columns:
    print(s)

In [ ]:
train, test = train_test_split(source_df, test_size=0.1, random_state=42)
train_data: pd.DataFrame = TabularDataset(train)
test_data: pd.DataFrame = TabularDataset(test)
model=TabularPredictor(
    label='money_room',
    problem_type='regression',
    eval_metric='mape',
)
predictor=model.fit(
    train_data=train_data,
    test_data=test_data,
    presets='extreme_quality',
    time_limit=3600*2,
    num_gpus=1
)

No path specified. Models will be saved in: "AutogluonModels\ag-20260109_023034"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.10.9
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          16
Pytorch Version:    2.7.1+cpu
CUDA Version:       CUDA is not available
Memory Avail:       31.06 GB / 63.74 GB (48.7%)
Disk Space Avail:   102.95 GB / 930.87 GB (11.1%)
Presets specified: ['extreme_quality']
Using hyperparameters preset: hyperparameters='zeroshot_2025_12_18_gpu'
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ... Time limit = 7200s
AutoGluon will save models to "c:\Users\it160\OneDrive\ドキュメント\my\signature\my_code\AutogluonModels\ag-20260109_023034"
Train Data Rows:    327531
Train Data Columns: 181
Label Column:       money_room
Problem Type:       regression
Preprocessing data

In [20]:
predictor.leaderboard(test)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-0.131990,-0.132345,mean_absolute_percentage_error,224.281649,2748.224261,21958.580598,0.006318,0.006143,0.758601,2,True,8
1,LightGBM_r163_BAG_L1,-0.137801,-0.142643,mean_absolute_percentage_error,187.091381,2694.900823,3277.296359,187.091381,2694.900823,3277.296359,1,True,5
2,LightGBMPrep_r13_BAG_L1,-0.140802,-0.140696,mean_absolute_percentage_error,29.394769,44.043167,14608.483246,29.394769,44.043167,14608.483246,1,True,2
3,NeuralNetTorch_r37_BAG_L1,-0.152944,-0.165974,mean_absolute_percentage_error,7.789180,9.274127,4072.042392,7.789180,9.274127,4072.042392,1,True,4
4,LightGBM_r177_BAG_L1,-0.154279,-0.156750,mean_absolute_percentage_error,88.365611,1157.792774,1455.177033,88.365611,1157.792774,1455.177033,1,True,3
5,NeuralNetFastAI_r25_BAG_L1,-0.221135,-0.247811,mean_absolute_percentage_error,4.122883,9.450986,287.707822,4.122883,9.450986,287.707822,1,True,6
6,CatBoost_c1_BAG_L1,-0.470599,-0.469689,mean_absolute_percentage_error,2.214846,4.204989,4402.003007,2.214846,4.204989,4402.003007,1,True,1
7,LightGBMPrep_r41_BAG_L1,-0.705956,-0.708931,mean_absolute_percentage_error,16.027368,12.278573,300.820014,16.027368,12.278573,300.820014,1,True,7


In [15]:
predictor.leaderboard(test)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-0.133033,-0.133513,mean_absolute_percentage_error,27.010612,1.179119,552.032809,0.032720,0.000000,0.026354,2,True,3
1,LightGBMXT,-0.133145,-0.133622,mean_absolute_percentage_error,22.606178,0.954127,420.042369,22.606178,0.954127,420.042369,1,True,1
2,LightGBM,-0.149793,-0.150833,mean_absolute_percentage_error,4.371713,0.224992,131.964086,4.371713,0.224992,131.964086,1,True,2


In [34]:
predictor.leaderboard(test)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-0.138503,-0.136393,mean_absolute_percentage_error,14.718167,0.685139,966.090227,0.035507,0.000000,0.024002,2,True,4
1,LightGBM,-0.145050,-0.143638,mean_absolute_percentage_error,9.878020,0.427611,318.002412,9.878020,0.427611,318.002412,1,True,2
2,LightGBMXT,-0.146452,-0.145826,mean_absolute_percentage_error,3.994979,0.184530,181.997473,3.994979,0.184530,181.997473,1,True,1
3,RandomForestMSE,-0.174417,-0.172940,mean_absolute_percentage_error,0.809661,0.072998,466.066339,0.809661,0.072998,466.066339,1,True,3


In [53]:
len(test.sample(n=1000))

1000

In [17]:
test_df = pd.read_csv(os.path.join('..', 'data', 'test', 'test.csv'))
test_gdf=utils.convert_to_geodataframe(test_df)
test_gdf=merge_lag_features(test_gdf,gpd.read_file(os.path.join('..', 'data', 'train', 'L01-24.geojson')),"ID","_kouji",map1)
test_gdf=merge_lag_features(test_gdf,gpd.read_file(os.path.join('..', 'data', 'train', 'L02-24.geojson')),"ID","_tyousa",map2)
test_gdf=test_gdf.drop(columns=["distance"])
test_gdf=utils.nearest_merge(test_gdf,gdf_json,threshold=1000)

result_gdf = utils.join_nearest_with_conditions(test_gdf, rosen[["ID","geometry","eki_name1"]].reset_index(drop=True), match_col='eki_name1', max_dist=100000.0)
result_gdf=result_gdf.drop(columns=["index__target","eki_name1_"])
result_gdf=result_gdf.rename(columns={"eki_name1__target":"eki_name1"})
result_gdf=utils.merge_lag_features(result_gdf,rosen,"ID","_eki1",map3)
result_gdf=result_gdf.drop(columns=["ID"])
rosen=rosen.rename(columns={"eki_name1":"eki_name2"})
result_gdf = utils.join_nearest_with_conditions(result_gdf, rosen[["ID","geometry","eki_name2"]].reset_index(drop=True), match_col='eki_name2', max_dist=100000.0)
result_gdf=result_gdf.drop(columns=["index__target","eki_name2_"])
result_gdf=result_gdf.rename(columns={"eki_name2__target":"eki_name2"})
result_gdf=utils.merge_lag_features(result_gdf,rosen,"ID","_eki2",map3)
result_gdf=result_gdf.drop(columns=["ID"])

test_gdf=result_gdf.loc[:, ~result_gdf.columns.duplicated()]
test_gdf=test_gdf.drop(columns=["target_ym"])

test_gdf["land_price_x"]=test_gdf.geometry.x
test_gdf["land_price_y"]=test_gdf.geometry.y
test_gdf = pd.DataFrame(test_gdf.drop(columns=[col for col in test_gdf.columns if 'geometry' in col]))

C:\Users\it160\AppData\Local\Temp\ipykernel_139180\2030608075.py:1: DtypeWarning: Columns (46,55,56,63,146) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv(os.path.join('..', 'data', 'test', 'test.csv'))


In [18]:
test_data: pd.DataFrame = TabularDataset(test_gdf)
predictions = predictor.predict(test_data)

In [19]:
pred=pd.DataFrame(predictions)
pred.to_csv(os.path.join('..', 'data', 'submit', 'iwasaki_submit.csv'), index_label='id')

In [13]:
importance_df = predictor.feature_importance(test.sample(n=1000), num_shuffle_sets=1)

Computing feature importance via permutation shuffling for 181 features using 5000 rows with 5 shuffle sets...
	40839.15s	= Expected runtime (8167.83s per shuffle set)


KeyboardInterrupt: 

In [ ]:
# カラム名(インデックス)に数字を含む行を除外
importance_df_d = importance_df[importance_df.index.astype(str).str.contains(r'\d', regex=True)]

In [36]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(importance_df)

,importance,stddev,p_value,n,p99_high,p99_low
house_area,2.779347e-01,0.012105,4.307423e-07,5,0.302860,0.253010
L01_008,2.703521e-01,0.006758,4.680146e-08,5,0.284266,0.256438
year_built,1.833584e-01,0.006122,1.488602e-07,5,0.195963,0.170754
L01_001,1.045423e-01,0.008737,5.799574e-06,5,0.122532,0.086553
snapshot_land_area,3.457181e-02,0.002758,4.821122e-06,5,0.040251,0.028893
lag_1_val__tyousa,3.413345e-02,0.001255,2.191261e-07,5,0.036718,0.031549
madori_kind_all,2.188616e-02,0.002007,8.388497e-06,5,0.026018,0.017754
rosen_name1,1.882773e-02,0.002844,6.058173e-05,5,0.024683,0.012973
eki_name1,1.532626e-02,0.001347,7.080729e-06,5,0.018099,0.012553
L01_004,1.327003e-02,0.002013,6.169594e-05,5,0.017416,0.009124


In [ ]:
drop=["reform_place_other","reform_common_area","parking_keiyaku","est_other_distance",'building_name_ruby', 'name_ruby', 'school_ele_code', 'school_jun_code', 'money_hoshou_company', 'free_rent_duration', 'free_rent_gen_timing',]